<a href="https://colab.research.google.com/github/MathMachado/eDreams/blob/master/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import main Python libraries



In [0]:
!pip install bamboolib

In [0]:
import pandas as pd
import numpy as np

import matplotlib
import bamboolib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
url_train= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/train.csv?token=AGDJQ67D2WENLE2YEMFJBPC5SHK26"
url_test= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/test.csv?token=AGDJQ6YYHJOGUATPKWUASIK5SSGF6"

# empilhando as amostras de treinamento e validação para fazermos um único tratamento
df_train= pd.read_csv(url_train, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])
df_test= pd.read_csv(url_test, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])

# redefinindo os índices da amostra de teste
df_test.index= range(50000, 80000)

# merge train and test
df = df_train.append(df_test, sort= True)

# Registra os índices do dataframe de treinamento e teste para separarmos estes dataframes mais tarde
train_index = df_train.index
test_index = df_test.index

In [0]:
df.shape

In [0]:
df.head()

In [0]:
df.tail()

In [0]:
df_test.head()

In [0]:
df_test.tail()

In [0]:
df.info()

In [0]:
# Treating date type columns
df2= df.copy()
df2['DEPARTURE_2']= df2['DEPARTURE'] +'/2018'
df2['ARRIVAL_2']= df2['ARRIVAL'] +'/2018'
df2['ARRIVAL_3']= df2['ARRIVAL'] +'/2019'

df2['DEPARTURE_2']= pd.to_datetime(df2['DEPARTURE_2'])
df2['ARRIVAL_2']= pd.to_datetime(df2['ARRIVAL_2'])
df2['ARRIVAL_3']= pd.to_datetime(df2['ARRIVAL_3'])
df2.head()

In [0]:
# Fixing ARRIVAL when ARRIVAL < DEPARTURE
df3= df2.copy()
df3['IS_ARRIVAL_BEFORE_DEPARTURE']= df3['ARRIVAL_2']<df3['DEPARTURE_2']
df3.loc[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True, 'ARRIVAL_2']= df3['ARRIVAL_3']

In [0]:
df3.head()

In [0]:
# Calculating the differente between ARRIVAL and DEPARTURE
df3['ARRIVAL_DEPARTURE']= (df3['ARRIVAL_2']-df3['DEPARTURE_2']).dt.days.astype(int)
df3[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True].head()

Algo estranho com a variável ARRIVAL e DEPARTURE, pois não faz muito sentido, pelo menos para mim, alguém sair no dia 07/07 e chegar no dia 06/fevereiro (index= 489. São 152 dias! Observe, por exemplo, o indice= 576, com 270 dias que separa DEPARTURE de ARRIVAL.

In [0]:
sns.distplot(df3['ARRIVAL_DEPARTURE'])

In [0]:
df3_Zoom= df3[df3['ARRIVAL_DEPARTURE']> 5]
df3_Zoom.shape[0]

Muitos registros com ARRIVAL_DEPARTURE > 5, o que não faz sentido pra mim! Uma viagem durar mais que 5 dias!?

In [0]:
# Deletando variáveis não-necessárias
df3= df3.drop(columns= ['DEPARTURE','ARRIVAL','ARRIVAL_3', 'TIMESTAMP'], axis= 1)

As we can see above, all rows of the dataframe refers to July.

In [0]:
df3.info()

Aparentemente, temos alguns problemas de Missing Values para DEVICE. Não se preocupe com os Missing values da variável EXTRA_BAGGAGE que é nossa variável-resposta e os 30.000 Missing values apresentados vem da amostra de teste e são justamente os valores que queremos prever.

In [0]:
df3.head()

In [0]:
df3= df3.drop(columns= ['DEPARTURE_2','ARRIVAL_2'], axis= 1)

In [0]:
# Converting column DISTANCE to numeric. For this purpose, I'll cut the distance in the ","

df3[['DISTANCE_2','DISTANCE_REST']] = df3['DISTANCE'].str.split(",",expand=True)
df3['DISTANCE_2']= pd.to_numeric(df3['DISTANCE_2'])
df3[['HAUL_TYPE','DISTANCE','DISTANCE_2','DISTANCE_REST']].head(10)

In [0]:
df3.groupby('HAUL_TYPE').agg({'DISTANCE_2': ['min', 'median', 'max', 'count']})

Alguma coisa errada com os valores mínimos. Não faz sentido DOMESTIC= 0. Muito menos INTERCONTINENTAL= 0. Vamos investigar um pouco mais isso a seguir. No entanto, vou trabalhar com DISTANCE_2 e desconsiderar DISTANCE_REST.

In [0]:
df4= df3.copy()
df4= df4.drop(columns= ['DISTANCE_REST','DISTANCE', 'IS_ARRIVAL_BEFORE_DEPARTURE'], axis= 1)
df4= df4.rename({'DISTANCE_2': 'DISTANCE'}, axis=1)
df4.head()

In [0]:
# Investigando DISTANCE = 0
df4[df4['DISTANCE']==0].count()

Há 288 registros em que DISTANCE= 0. Eu considero estes registros como Missing Values. Como tratamento, proponho atribuir à estes casos a mediana segmentado por HAUL_TYPE. Ou seja

In [0]:
median_by__HAUL_TYPE= df4.groupby('HAUL_TYPE')['DISTANCE'].median()
median_by__HAUL_TYPE

In [0]:
# Median for DOMESTIC --> Vou usar este valor para os missing values de DISTANCE quando HAUL_TYPE= 'DOMESTIC'
median_by__HAUL_TYPE[1]

In [0]:
# Median for DOMESTIC --> Vou usar este valor para os missing values de DISTANCE quando HAUL_TYPE= 'INTERCONTINENTAL'
median_by__HAUL_TYPE[2]

Vou atribuir o valor 1237 para os Missing values do CONTINENTAL, 625 para os MV do DOMESTIC e 4051 para os MV do INTERCONTINENTAL.

In [0]:
# Identificando os MV. No caso, zeros
df4.loc[df4['DISTANCE'] == 0, 'DISTANCE']= np.nan

# Checking Missing Values
df4.isna().sum()

Vamos tratar os MV da coluna DISTANCE  e DEVICE a seguir:

In [0]:
# Missing Value imputation for DOMESTIC
df4['DISTANCE'] = np.where(((df4['DISTANCE'].isnull()) & (df4['HAUL_TYPE'] =="DOMESTIC")), median_by__HAUL_TYPE[1], df4['DISTANCE'])

# Missing Value imputation for INTERCONTINENTAL
df4['DISTANCE'] = np.where(((df4['DISTANCE'].isnull()) & (df4['HAUL_TYPE'] =="INTERCONTINENTAL")), median_by__HAUL_TYPE[2], df4['DISTANCE'])

df4.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
# Checking Missing Values
df4.isna().sum()

In [0]:
# Treating Missing Values of DEVICE
df4['DEVICE'].value_counts() 

In [0]:
# Replacing NaN's of DEVICE with 'NO DEVICE'
df4["DEVICE"].fillna("NO DEVICE", inplace= True)

# Checking Missing Values
df4.isna().sum()

Como podemos observar acima, os missing values foram tratados.

In [0]:
# Treating Missing Values of DEVICE
df4['DEVICE'].value_counts() 

In [0]:
df5= df4.copy()
df5.head()

# Tratando os outliers
Considere o output a seguir:

In [0]:
# Min, Max e Median ANTES do tratamento dos outliers
df5.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
sns.distplot(df5['DISTANCE'])

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='DISTANCE', kind="box", data=df5, height=5, aspect=2)
plt.show()

## Detecting Outliers with Z-Score
> Wikipedia Definition
>> The Z-score is the signed number of standard deviations by which the value of an observation or data point is above the mean value of what is being observed or measured.

Em outras palavras: The Z-score is a simple calculation that answers the question, “Given a data point, how many standard deviations is it away from the mean?

In [0]:
from scipy import stats
threshold = 3

# Z-Score para HAUL_TYPE== 'DOMESTIC'
df_DOMESTIC= df5[df5['HAUL_TYPE']== 'DOMESTIC']

# Z-Score para HAUL_TYPE== 'CONTINENTAL'
df_CONTINENTAL= df5[df5['HAUL_TYPE']== 'CONTINENTAL']

# Z-Score para HAUL_TYPE== 'INTERCONTINENTAL'
df_INTERCONTINENTAL= df5[df5['HAUL_TYPE']== 'INTERCONTINENTAL']

In [0]:
df_DOMESTIC['HAUL_TYPE'].value_counts() 

In [0]:
df_CONTINENTAL['HAUL_TYPE'].value_counts() 

In [0]:
df_INTERCONTINENTAL['HAUL_TYPE'].value_counts() 

In [0]:
# Lista de todas as linhas onde HAUL_TYPE== 'DOMESTIC' e ZS_DOMESTIC > threshold
ZS_DOMESTIC = np.abs(stats.zscore(df_DOMESTIC['DISTANCE']))

# Lista de todas as linhas onde HAUL_TYPE== 'CONTINENTAL' e ZS_CONTINENTAL > threshold
ZS_CONTINENTAL = np.abs(stats.zscore(df_CONTINENTAL['DISTANCE']))

# Lista de todas as linhas onde HAUL_TYPE== 'INTERCONTINENTAL' e ZS_INTERCONTINENTAL > threshold
ZS_INTERCONTINENTAL = np.abs(stats.zscore(df_INTERCONTINENTAL['DISTANCE']))

In [0]:
# IQR-Score
Q1_DOMESTIC = df_DOMESTIC['DISTANCE'].quantile(0.25)
Q3_DOMESTIC = df_DOMESTIC['DISTANCE'].quantile(0.75)
IQR_DOMESTIC = Q3_DOMESTIC - Q1_DOMESTIC

Q1_CONTINENTAL = df_CONTINENTAL['DISTANCE'].quantile(0.25)
Q3_CONTINENTAL = df_CONTINENTAL['DISTANCE'].quantile(0.75)
IQR_CONTINENTAL = Q3_CONTINENTAL - Q1_CONTINENTAL

Q1_INTERCONTINENTAL = df_INTERCONTINENTAL['DISTANCE'].quantile(0.25)
Q3_INTERCONTINENTAL = df_INTERCONTINENTAL['DISTANCE'].quantile(0.75)
IQR_INTERCONTINENTAL = Q3_INTERCONTINENTAL - Q1_INTERCONTINENTAL

In [0]:
# Índices baseados no Z-Score
Idx_DOMESTIC= np.where(ZS_DOMESTIC > threshold)
Idx_CONTINENTAL= np.where(ZS_CONTINENTAL > threshold)
Idx_INTERCONTINENTAL= np.where(ZS_INTERCONTINENTAL > threshold)

Todos os índices em Idx_DOMESTIC, Idx_CONTINENTAL e Idx_INTERCONTINENTAL possuem z-score > 3.

In [0]:
# removing outliers based on ZS_DOMESTIC
df6= df5.drop(df5.index[Idx_DOMESTIC])
df6.shape[0]

In [0]:
# Número de linhas a serem deletadas devido ZS_DOMESTIC
np.shape(Idx_DOMESTIC)[1]

In [0]:
# removing outliers based on ZS_CONTINENTAL
df7= df6.drop(df6.index[Idx_CONTINENTAL])
df7.shape[0]

In [0]:
# Número de linhas a serem deletadas devido ZS_CONTINENTAL
np.shape(Idx_CONTINENTAL)[1]

In [0]:
# removing outliers based on ZS_INTERCONTINENTAL
df8= df7.drop(df7.index[Idx_INTERCONTINENTAL])
df8.shape[0]

In [0]:
# Número de linhas a serem deletadas devido ZS_INTERCONTINENTAL
np.shape(Idx_INTERCONTINENTAL)[1]

In [0]:
# Número de instâncias deletadas devido aos outliers
80000-78794

In [0]:
# Min, Max e Median ANTES do tratamento dos outliers
df5.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
# Min, Max e Median DEPOIS do tratamento dos outliers
df8.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

# Outliers pelo IQR-Score

In [0]:
df_DOMESTIC.shape[0]

In [0]:
# Índices baseados no IQR-Score
df_DOMESTIC_IQR = df_DOMESTIC[~((df_DOMESTIC['DISTANCE'] < (Q1_DOMESTIC-1.5*IQR_DOMESTIC)) |(df_DOMESTIC['DISTANCE'] > (Q3_DOMESTIC+1.5*IQR_DOMESTIC)))]
df_DOMESTIC_IQR.shape[0]

In [0]:
df_CONTINENTAL.shape[0]

In [0]:
df_CONTINENTAL_IQR = df_CONTINENTAL[~((df_CONTINENTAL['DISTANCE'] < (Q1_CONTINENTAL-1.5*IQR_CONTINENTAL)) |(df_CONTINENTAL['DISTANCE'] > (Q3_CONTINENTAL+1.5*IQR_CONTINENTAL)))]
df_CONTINENTAL_IQR.shape[0]

In [0]:
df_INTERCONTINENTAL.shape[0]

In [0]:
df_INTERCONTINENTAL_IQR = df_INTERCONTINENTAL[~((df_INTERCONTINENTAL['DISTANCE'] < (Q1_INTERCONTINENTAL-1.5*IQR_INTERCONTINENTAL)) |(df_INTERCONTINENTAL['DISTANCE'] > (Q3_INTERCONTINENTAL+1.5*IQR_INTERCONTINENTAL)))]
df_INTERCONTINENTAL_IQR.shape[0]

In [0]:
df_IQR = df_DOMESTIC_IQR.append(df_CONTINENTAL_IQR, sort= True)
df_IQR = df_IQR.append(df_INTERCONTINENTAL_IQR, sort= True)
df_IQR.shape[0]

In [0]:
80000-76135

In [0]:
df_IQR.head()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='DISTANCE', kind="box", data=df_IQR, height=5, aspect=2)
plt.show()

Distribuição da variável-resposta depois do tratamento de outliers pelo IQR:

In [0]:
# Variável resposta - ANTES
df5['EXTRA_BAGGAGE'].value_counts() 

In [0]:
# Variável resposta - DEPOIS
df_IQR['EXTRA_BAGGAGE'].value_counts() 

Perda pequena... Portanto, vou continuar com o banco de dados com tratamento para outliers pelo IQR.

In [0]:
# Min, Max e Median ANTES do tratamento dos outliers
df5.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
# Min, Max e Median DEPOIS do tratamento dos outliers
df_IQR.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

Observe que a mediana mudou um pouco quando se compara antes e depois do tratamento dos outliers. De todo modo, vou continuar assim mesmo, pois não vejo como problema...

In [0]:
sns.distplot(df_IQR['DISTANCE'])

# Binning variáveis numéricas

In [0]:
df9= df_IQR.copy()
# Variável resposta
df9['EXTRA_BAGGAGE'].value_counts() 

In [0]:
df9['EXTRA_BAGGAGE']= 1*df9['EXTRA_BAGGAGE']
df9.head()

In [0]:
df9.info()

### Tratando as variáveis numéricas

In [0]:
df9.isna().sum()

In [0]:
sns.distplot(df9['ARRIVAL_DEPARTURE'])

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='ARRIVAL_DEPARTURE', kind="box", data=df9, height=5, aspect=2)
plt.show()

In [0]:
df9['DISTANCE_CAT']= pd.cut(df9['DISTANCE'], 10)
df9['ARRIVAL_DEPARTURE_CAT']= pd.cut(df9['ARRIVAL_DEPARTURE'], 10)

In [0]:
df9.head()

In [0]:
ARRIVAL_DEPARTURE NEGATIVO????? NÃO PODE SER!!!